In [2]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import holidays
import datetime
%matplotlib inline

In [3]:
!pwd; ls -ll;

pwd: unknown option -- l
Try 'pwd --help' for more information.


2 Cells below is relevant only when activating on cloud / Linux machine.
Since on Windows  google.colab not working


from google.colab import files
uploaded = files.upload()

import io
DataFrame = pd.read_csv(io.BytesIO(uploaded['Crimes_-_2020.csv']))
DataFrame

Update csv data bease:
https://drive.google.com/file/d/1s5lavM_C9aLsgdFzUmXOfbh_XvSxl-G4/view?usp=sharing

In [4]:
DataFrame = pd.read_csv("Crimes_-_2019_to_Present.csv")

In [19]:
Sunrise = pd.read_csv('sunrise-chicago.csv')

# Add converted columns
Sunrise['dtsunset']=Sunrise.apply(lambda x: datetime.datetime.combine(pd.to_datetime(x['date']), pd.to_datetime(x['sunset']).time()), axis=1)
Sunrise['dtsunrise']=Sunrise.apply(lambda x: datetime.datetime.combine(pd.to_datetime(x['date']), pd.to_datetime(x['sunrise']).time()), axis=1)

# a constant defining the time between sunset and darkness
delta_sunset = datetime.timedelta(0,3600)
#TODO: check exact delta, set another constant for sunrise time

# Accepts datetime in string, returns boolean if it's in night time
def is_dark(datecol) :
  d = pd.to_datetime(datecol)
  row = Sunrise.loc[Sunrise['date']==d.strftime("%Y-%m-%d")]
  return d < row['dtsunrise'].item() or d > (row['dtsunset'].item() + delta_sunset)

# Add IsDark colunm to DataFrame
# for tests, uncomment the next line and comment the line after it
#DataFrame = DataFrame.head()
DataFrame['IsDark']=DataFrame.apply(lambda x: is_dark(x['Date']) ,axis=1)


Based on link below, we shall use FBI code instead of IUCR .
http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html#N04A
For example :

> **FBI code** - Homicide 1st & 2nd Degree - 01A

> **IUCR** 
*  0110 HOMICIDE FIRST DEGREE MURDER
*  0130 HOMICIDE SECOND DEGREE MURDER

> **FBI code** - Nonvoluntary Manslaughter - 01B

> **IUCR** 
*    0141 HOMICIDE INVOLUNTARY MANSLAUGHTER
*    0142 HOMICIDE RECKLESS HOMICIDE

In [5]:
DataFrame.columns

Index([u'Unnamed: 0', u'ID', u'Case Number', u'Date', u'Block', u'IUCR',
       u'Primary Type', u'Description', u'Location Description', u'Arrest',
       u'Domestic', u'Beat', u'District', u'Ward', u'Community Area',
       u'FBI Code', u'X Coordinate', u'Y Coordinate', u'Year', u'Updated On',
       u'Latitude', u'Longitude', u'Location'],
      dtype='object')

**dt0 - Seasons numeric**

*   4- Fall   - Lowest crime rate
*   3- Summer - Highest crime rate

*   2- Spring
*   1- Winter

**dt1 - Weekday numeric**
 The day of the week with Monday=0, Sunday=6 
 
**dt2 - Weekday string** 

*  Sunday - Highest rate crimes
*  Tuesday - Lowest rate crimes


   
    


In [6]:
for date in holidays.UnitedStates(years=2020).items():print(str(date[0]))

2020-12-25
2020-07-03
2020-11-26
2020-07-04
2020-10-12
2020-05-25
2020-01-20
2020-11-11
2020-09-07
2020-01-01
2020-02-17


In [7]:
us_holidays = []
for date in holidays.UnitedStates(years=2020).items():
    us_holidays.append(str(date[0]))
print(us_holidays)

['2020-12-25', '2020-07-03', '2020-11-26', '2020-07-04', '2020-10-12', '2020-05-25', '2020-01-20', '2020-11-11', '2020-09-07', '2020-01-01', '2020-02-17']


In [8]:
dt0 = DataFrame.copy()
dt0.drop(['ID', 'Case Number', 'Updated On', 'Block', 'IUCR', 'Description', 'Domestic','Arrest', 'Beat', 'Location', 'X Coordinate', 'Y Coordinate', 'Ward', 'FBI Code'], 1, inplace=True)
dt0["Season"] = (pd.to_datetime(dt0['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.month%12 + 3)//3
dt0["Weekday"] = pd.to_datetime(dt0['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.dayofweek
dt0["Just_Date"] = (pd.to_datetime(dt0['Date'])).dt.date
dt0["Is_Holiday"] = [ 1 if str(val).split()[0] in us_holidays else 0 for val in dt0["Just_Date"]]
dt0["Day_of_Month"] = pd.to_datetime(dt0["Just_Date"]).dt.day
dt0["Month"] = pd.to_datetime(dt0["Just_Date"]).dt.month
#dt0["Day_of_Month"] = pd.to_datetime(dt0['Just_Date']).dt.day.values
##dt0["just_date"] = dt0['Date'].dt.date
##  dt0["Date"] = (pd.to_datetime(dt0['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.month%12 + 3)//3
## dt0["Date"].value_counts()
dt0.columns

Index([u'Unnamed: 0', u'Date', u'Primary Type', u'Location Description',
       u'District', u'Community Area', u'Year', u'Latitude', u'Longitude',
       u'Season', u'Weekday', u'Just_Date', u'Is_Holiday', u'Day_of_Month',
       u'Month'],
      dtype='object')

In [9]:
dt0

,Unnamed: 0,Date,Primary Type,Location Description,District,Community Area,Year,Latitude,Longitude,Season,Weekday,Just_Date,Is_Holiday,Day_of_Month,Month
0,136,03/19/2019 04:30:00 PM,THEFT,GROCERY FOOD STORE,17.0,13.0,2019,NaN,NaN,2,1,2019-03-19,0,19,3
1,148,03/19/2019 09:00:00 AM,THEFT,STREET,12.0,31.0,2019,NaN,NaN,2,1,2019-03-19,0,19,3
2,258,02/11/2019 05:50:00 PM,THEFT,RESIDENTIAL YARD (FRONT/BACK),14.0,22.0,2019,NaN,NaN,1,0,2019-02-11,0,11,2
3,279,03/20/2019 07:00:00 PM,CRIMINAL DAMAGE,STREET,24.0,2.0,2019,NaN,NaN,2,2,2019-03-20,0,20,3
4,286,03/20/2019 01:00:00 AM,THEFT,OTHER,24.0,77.0,2019,NaN,NaN,2,2,2019-03-20,0,20,3
5,298,03/20/2019 01:30:00 PM,DECEPTIVE PRACTICE,OTHER,25.0,25.0,2019,NaN,NaN,2,2,2019-03-20,0,20,3
6,342,02/01/2019 01:00:00 AM,DECEPTIVE PRACTICE,NaN,17.0,16.0,2019,NaN,NaN,1,4,2019-02-01,0,1,2
7,345,03/31/2019 01:00:00 AM,THEFT,BAR OR TAVERN,18.0,8.0,2019,NaN,NaN,2,6,2019-03-31,0,31,3
8,533,03/31/2019 11:00:00 AM,THEFT,HOTEL/MOTEL,1.0,32.0,2019,NaN,NaN,2,6,2019-03-31,0,31,3
9,707,03/21/2019 12:00:00 PM,THEFT,APARTMENT,12.0,24.0,2019,NaN,NaN,2,3,2019-03-21,0,21,3


In [10]:
import requests
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="My_geolocate")

lookingFor = "Chicago"
locations = geolocator.geocode(lookingFor)
type(locations)



##print(locations)
##r = requests.get('https://api.sunrise-sunset.org/json', params={'lat': dt0[Latitude], 'lng': dt0[Longitude]}).json()['results']
##r = requests.get('https://api.sunrise-sunset.org/json', params={'lat': locations.latitude, 'lng': locations.longitude}).json()['results']
##print('Sunrise:', r['sunrise'])
##print('Sunset:', r['sunset'])

ImportError: No module named geopy.geocoders

In [17]:
community_area_dict = dict()

for i in range(1,78):
    area_df = DataFrame[DataFrame["Community Area"]==i]
    vc = area_df["Block"].value_counts()
    max_block = vc.head(1).index[0]
    block_df = area_df[area_df["Block"]==max_block]
    var = block_df["Location"].iloc[0]
    count = 1
    while pd.isnull(var):
        var = block_df["Location"].iloc[count]
        count+=1
    community_area_dict[i] = var

{1: '(42.019399237, -87.675049485)',
 2: '(41.990615196, -87.683188577)',
 3: '(41.963070794, -87.655984213)',
 4: '(41.977017622, -87.692447486)',
 5: '(41.943552694, -87.688162342)',
 6: '(41.943122315, -87.649365938)',
 7: '(41.922786529, -87.645045119)',
 8: '(41.897895128, -87.624096605)',
 9: '(41.997177742, -87.807138486)',
 10: '(41.988702804, -87.81297224)',
 11: '(41.97008868, -87.762690091)',
 12: '(41.994019965, -87.748335679)',
 13: '(41.994758989, -87.7144777)',
 14: '(41.968235322, -87.728715018)',
 15: '(41.953443342, -87.744957048)',
 16: '(41.946542477, -87.736084376)',
 17: '(41.952897791, -87.787860507)',
 18: '(41.923817959, -87.805033506)',
 19: '(41.918711651, -87.76551063)',
 20: '(41.931580848, -87.742956658)',
 21: '(41.945238279, -87.712670479)',
 22: '(41.929743818, -87.684273777)',
 23: '(41.895431459, -87.719481487)',
 24: '(41.910006508, -87.676887355)',
 25: '(41.909664252, -87.742728815)',
 26: '(41.880751096, -87.723855981)',
 27: '(41.880991005, -87.7

In [ ]:
from heliacal import Heliacal
Chicago = Heliacal(dt0[Latitude], dt0[Longitude])

In [ ]:
dt1 = DataFrame.copy()
dt1["Arrest"] = dt1["Arrest"].astype(int)
dt1["Domestic"] = dt1["Domestic"].astype(int)
dt1["FBI Code"] = (DataFrame["FBI Code"]).apply(lambda x: int(x, 16))
dt1["Date"] = pd.to_datetime(dt1['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.dayofweek
dt1

In [ ]:
dt0["Month"].value_counts()

Should we  remove rows related to 6 days at the beginning of September?

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Day_of_Month"]]
dt0.plot(kind='scatter', x="FBI Code", y='Day_of_Month', alpha=(0.4),figsize=(30,5))
plt.show()

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Date"]]
dt0.plot(kind='scatter', x="FBI Code", y='Date', alpha=(0.4),figsize=(30,5))
plt.show()

In [ ]:
codt1=dt1[dt1["Date"]==6]
codt1.head(3)

In [ ]:
codt1.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(20,20),c="Date", cmap=plt.get_cmap("Blues"), colorbar=True, sharex=False)
plt.legend()

In [ ]:
dt2 = DataFrame.copy()
dt2['Date'] = pd.to_datetime(dt2['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.day_name()
dt2

In [ ]:
dt2["Date"].value_counts()

In [ ]:
DataFrame.shape[0]

In [ ]:
DataFrame.shape[1]

In [ ]:
DataFrame["Community Area"].value_counts()

Community Area no. 25 appears 8722 time in the data set

In [ ]:
co=DataFrame[DataFrame["Community Area"]==25]
co.head(3)

"Block" crimes distribution on "Community Area" no.25 It is seen that the top 3 streets with crime:

    052XX W MADISON ST(72) & 056XX W MADISON ST(49) & 048XX W MADISON ST(47)
    001XX N CENTRAL AVE(63) & 003XX N CENTRAL AVE(58)
    003XX S CICERO AVE(60) & 007XX S CICERO AVE(50)



In [ ]:
co["Block"].value_counts().head(10)

"Primary Type" crimes distribution on "Community Area" no.25. It is seen that the top 4 crimes:

    Battery
    Theft
    Criminal Damage
    Assualt



In [ ]:
co["Primary Type"].value_counts()

In [ ]:
DataFrame.Description.value_counts()

In [ ]:
DataFrame["Location Description"].value_counts()

In [ ]:
DataFrame.sort_values(by=["Location Description", "Primary Type"])

In [ ]:
DataFrame[(DataFrame["Primary Type"]=="ASSAULT")&(DataFrame["Location Description"]=="STREET")]

In [ ]:
DataFrame.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(50,50),c="Community Area", cmap=plt.get_cmap("jet"), colorbar=True, sharex=False)
plt.legend()

In [ ]:
co.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(10,10),c="Community Area", cmap=plt.get_cmap("Blues"), colorbar=True, sharex=False)
plt.legend()

Plot histogram that shows per district the number of total crimes.
Next step would be to create a testing data frame , from 20% of the dataset.
Need to make sure that it would resemble  the distribution of all DataSet

29/10/20 START

In [ ]:
dt0.describe()

In [ ]:
corrMatrix= dt0.corr()
corrMatrix["FBI Code"].sort_values(ascending=True)

29/10/20 What is the most common crime  (6 is the most common crime which is theft)

In [ ]:
dt0["Community Area"].value_counts()

In [ ]:
dt0["FBI Code"].value_counts()

In [ ]:
for6=dt0[dt0["FBI Code"]==6]
for6

In [ ]:
for6["Community Area"].value_counts()

From above area 8 has the highest theft crime. I expected based on below linear regression that 36 area would be the one

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Community Area"]]
dt0.plot(kind='scatter', x="FBI Code", y='Community Area', alpha=(0.4),figsize=(30,10))
plt.show()

In [ ]:
import sklearn.linear_model
model = sklearn.linear_model.LinearRegression()
model.fit(x, y)

In [ ]:
X_new = [[6]]

In [ ]:
print(model.predict(X_new))

In [ ]:
for36=dt0[dt0["Community Area"]==36]
for36

In [ ]:
for36["FBI Code"].value_counts()

In [ ]:
dt0["FBI Code"].value_counts()

In [ ]:
X_new = [[27]]

In [ ]:
print(model.predict(X_new))

29/10/20 END

In [ ]:
DataFrame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
from sklearn.model_selection import train_test_split
train_frame, test_frame = train_test_split(DataFrame)

In [ ]:
test_frame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
train_frame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
len(train_frame)

In [ ]:
len(test_frame)

Convert "FBI Code" Hexa string to numeric value

In [ ]:
dt1['FBI Code'] = (DataFrame["FBI Code"]).apply(lambda x: int(x, 16))
dt1

In [ ]:
from sklearn import linear_model
from sklearn.utils import shuffle

In [ ]:
dt0=dt0[["Date","Arrest","Domestic","Ward","Community Area","FBI Code"]]
dt0

In [ ]:
dt0.info()

In [ ]:
dt0[(dt0["Date"]=="NaN")]

In [ ]:
predict="FBI Code"
X=np.array(dt0.drop([predict],1))

In [ ]:
type(X)

In [ ]:
Y=np.array(dt0[predict])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.1)

In [ ]:
linear=linear_model.LinearRegression()

In [ ]:
dt0

In [ ]:
def clean_dataset(dt0):
    assert isinstance(dt0, pd.DataFrame), "df needs to be a pd.DataFrame"
    dt0.dropna(inplace=True)
    indices_to_keep = ~dt0.isin([np.nan, np.inf, -np.inf]).any(1)
    return dt0[indices_to_keep].astype(np.float64)

In [ ]:
dt0.dropna()

In [ ]:
linear.fit(x_train, y_train)
acc=linear.score(x_test, y_test)
print(acc)

In [ ]:
dt0[(dt0["Community Area"]=="NaN")]

In [ ]:
corrMatrix= dt1.corr()
corrMatrix["FBI Code"].sort_values(ascending=True)

In [ ]:
corrMatrix= dt0.corr()
corrMatrix["FBI Code"].sort_values(ascending=True)